In [6]:
# import modules
import numpy as np

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from scipy.integrate import odeint

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import adam

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Schrodinger function

In [8]:
class PhysicsInformedNN(nn.Module):
    def __init__(self, x0, u0, v0, tb, X_f, lb, ub, layers):
        """
        x0: T=0, initial condition, randomly drawn from the domain
        u0: the real part of the value
        v0: the imaginary part of the value
        tb: the time boundary
        X_f: the collocation points with time, size (Nf, 2)
        lb: the lower bound of the domain (x_min, t_min)
        ub: the upper bound of the domain (x_max, t_max)
        layers: the number of neurons in each layer
        """
        super(PhysicsInformedNN, self).__init__()

        # Data
        self.X0 = torch.tensor(np.concatenate((x0, 0), 1)) # (x, 0)
        self.X_f = torch.tensor(X_f)
        self.X_lb = torch.tensor(np.concatenate((lb[0], tb), axis=1))
        self.X_ub = torch.tensor(np.concatenate((ub[0], tb), axis=1))
        
        self.u0 = torch.tensor(u0)
        self.v0 = torch.tensor(v0)

        # Bounds
        self.lb = lb
        self.ub = ub
        
        # Initialize NNs
        self.layers = layers
        self.model = nn.Sequential()
        for l in range(0, len(layers) - 2):
            self.model.add_module("linear" + str(l), nn.Linear(layers[l], layers[l+1]))
            self.model.add_module("tanh" + str(l), nn.Tanh())
        self.model.add_module("linear" + str(len(layers) - 2), nn.Linear(layers[-2], layers[-1]))

    # calculate the function h(x, t) using neural nets
    def net_uv(self, x, t):
        X = torch.cat([x, t], dim=1)
        H = (X - self.lb) / (self.ub - self.lb) * 2.0 - 1.0 # normalize to [-1, 1]
        uv = self.model(H)
        u = uv[:, 0:1]
        v = uv[:, 1:2]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        return u, v, u_x, v_x

    # compute the Schrodinger function on the collacation points
    def net_f_uv(self, x, t):
        u, v, u_x, v_x = self.net_uv(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        return f_u, f_v


    def forward(self, x, t):
        u, v, f_u, f_v = self.net_f_uv(x, t)
        return u, v, f_u, f_v

    def loss_function(self):
        loss = nn.MSELoss()
        self.u0_pred, self.v0_pred, _, _ = self.net_uv(self.x0, self.t0)
        
        self.u_lb_pred, self.v_lb_pred, self.u_x_lb_pred, self.v_x_lb_pred = self.net_uv(self.X_lb[:, 0:1], self.X_lb[:, 1:2])
        self.u_ub_pred, self.v_ub_pred, self.u_x_ub_pred, self.v_x_ub_pred = self.net_uv(self.X_ub[:, 0:1], self.X_ub[:, 1:2])
        
        self.f_u_pred, self.f_v_pred = self.net_f_uv(self.X_f[:, 0:1], self.X_f[:, 1:2])
        
        # initial condition + boundary condition + PDE constraint
        MSE = loss(self.u0_pred, self.u0) + loss(self.v0_pred, self.v0) + \
            loss(self.u_lb_pred, self.u_ub_pred) + loss(self.v_lb_pred, self.v_ub_pred) + \
            loss(self.u_x_lb_pred, self.u_x_ub_pred) + loss(self.v_x_lb_pred, self.v_x_ub_pred) + \
            loss(self.f_u_pred, torch.zeros_like(self.f_u_pred)) + loss(self.f_v_pred, torch.zeros_like(self.f_v_pred))

        return MSE
    
    def train(self, epochs = 1e+4, lr = 1e-3):
        # Optimizer
        optimizer = adam.Adam(self.model.parameters(), lr=lr)
        
        # Training
        for epoch in tqdm(range(epochs)):
            optimizer.zero_grad()
            loss = self.loss_function()
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print('Epoch: %d, Loss: %.3e' % (epoch, loss.item()))

    def predict(self, X_star):
        u_star, v_star, f_u_star, f_v_star = self.forward(X_star[:, 0:1], X_star[:, 1:2])
        return u_star.detach().numpy(), v_star.detach().numpy(), f_u_star.detach().numpy(), f_v_star.detach().numpy()


In [9]:
import numpy as np
import scipy.io
from pyDOE import lhs
import time

# Hyperparameters
noise = 0.0        
lb = np.array([-5.0, 0.0]) # lower bound for [x, t]
ub = np.array([5.0, np.pi/2]) # upper bound for [x, t]
N0 = 50 # number of data for initial samples
N_b = 50 # number of data for boundary samples
N_f = 20000 # number of data for collocation points

# Define the physics-informed neural network
layers = [2, 100, 100, 100, 100, 2]

# Load data from simulated dataset
data = scipy.io.loadmat('./Data/NLS.mat')


t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_h = np.sqrt(np.real(Exact)**2 + np.imag(Exact)**2)
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
h_star = Exact_h.T.flatten()[:,None]
v_star = np.imag(Exact).T.flatten()[:,None]
u_star = np.real(Exact).T.flatten()[:,None]

# Initial and boundary data
idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x,:]
u0 = np.real(Exact[idx_x,0:1]) # or computed using h(0, x) = 2*sech(x)
v0 = np.imag(Exact[idx_x,0:1])
tb = t[np.random.choice(t.shape[0], N_b, replace=False),:] # random time samples

# Collocation points
X_f = lb + (ub-lb)*lhs(2, N_f)

# Train the model
model = PhysicsInformedNN(x0, u0, v0, tb, X_f, lb, ub, layers)
start_time = time.time()                
model.train(50000)
print('Training time: %.4f' % (time.time() - start_time))

# Predictions
u_pred, v_pred, _, _ = model.predict(X_star)
h_pred = np.sqrt(u_pred**2 + v_pred**2)
        
# Errors
errors = {'u': np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2),
          'v': np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2),
          'h': np.linalg.norm(h_star-h_pred,2)/np.linalg.norm(h_star,2)}
print('Errors: ', errors)

SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [4]:
import huggingface_hub.constants

if hasattr(huggingface_hub.constants, 'HF_HUB_CACHE'):
    print("huggingface_hub.constants has attribute HF_HUB_CACHE")
else:
    print("huggingface_hub.constants does not have attribute HF_HUB_CACHE")

huggingface_hub.constants does not have attribute HF_HUB_CACHE
